# Module 4: Portfolio Optimization

This notebook demonstrates portfolio optimization using the Markowitz model, with both sample and real financial data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize

In [ ]:
assets = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
data = yf.download(assets, start='2022-01-01', end='2023-01-01')['Close']
returns = data.pct_change().dropna()
returns.head()

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights) * 252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return returns, std

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    p_returns, p_std = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_returns - risk_free_rate) / p_std

In [ ]:
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_assets = len(assets)
args = (mean_returns, cov_matrix)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(num_assets))
result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=args,
                  method='SLSQP', bounds=bounds, constraints=constraints)
opt_weights = result.x
opt_weights

In [ ]:
# Display optimized weights
for asset, weight in zip(assets, opt_weights):
    print(f"{asset}: {weight:.2%}")

# Plot efficient frontier (simulation)
results = np.zeros((3, 10000))
for i in range(10000):
    weights = np.random.dirichlet(np.ones(num_assets), size=1)[0]
    p_ret, p_std = portfolio_performance(weights, mean_returns, cov_matrix)
    results[0,i] = p_std
    results[1,i] = p_ret
    results[2,i] = (p_ret - 0.01) / p_std
plt.figure(figsize=(10, 6))
plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='viridis', marker='o')
plt.xlabel('Volatility (Std Dev)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.colorbar(label='Sharpe Ratio')
plt.show()